# Goal
- Create a dataframe based on multiple JSON files
- Create "features" from both the findings and the tasks


#Loading NowSecure Files

1. You should have received a .zip file with a bunch of JSON files. Unzip these files and upload them. Click the folder icon to the left 
<----
2. Then click the ".." to navigate up to "content". Youc an see the whole file structure of the server that CoLab is running on. 

3. Then click the upload file icon and upload your json file(s). The current example is based on loading a single file. Make sure that your file uploads all the way before moving or renaming it -- otherwise the file may get corrupted.

4. Change the "/content/filename.json" path to match your uploaded file.

5. Run the cell below to load your files.

#Questions: 

1. Create a findings data frame where one of the columns is the `check_id` and another is the CVSS score
2. Generate an app-level data frame with columns `title` and a flag `weak_crypto` which is `True` if there is a finding `apk_weak_crypto_hashing_algorithms` for the app.
3. Identify other app level factors including the min/max/mean and count of CVSS score and the number of contacted URLs and add these to your data frame. You can use the below code from previous exercises.
4. Download your data frame as a .csv


This code loads all uploaded json files, then writes to `content/output/`. It's adapted from the snippet here: https://gist.github.com/jborowitz/c975bf12fc148fe2ed923bc77971cc1e with a change to run on colab.

This code takes files in the `/content/` directory that end in `*.json`. it then writes a file with 


In [ ]:
!mkdir -p /content/output/
'''
Author: Jeff Borowitz          
Take NowSecure data dumps as of 9/2021 and flatten them out a bit.  The goal
here is to take the schema from one list item per app to one list item per task
per app. This way it'd be possible to sort/filter the rows pretty easily and
combine only the per-task items into a file or ingest only the items for a
specific task type into an Elasticsearch or other schema discovering tool.
It should run from one folder above the json files, and will write output files
with the same name in a /output/ subdirectory, with a 'cleaned-' prefix.

Note that now these output files also contain rows with the NowSecure score as 
well as a couple findings.
'''
  
import glob
import json
  
for f in glob.glob('/content/*.json'):
    outlist = []        
    print(f)       
    try:
      doc  = json.load(open(f))  
    except json.JSONDecodeError as e:
      print('error parsing file: %s' % f)
    for app in doc:            
        out_stub = {}
        out_stub['title'] = app['title']
        assessment = app['assessment']  
        if assessment is None:
            continue
        out_stub['publisher'] = assessment['publisherName']
        out_stub['platformType'] = assessment['platformType']
        for task_type, value in assessment['analysis']['task'].items():
            out['assessment'] = assessment
            if value is None:
                continue
            if task_type == 'yaapStatic':   
                result_location = value['result'][0]
            else:
                result_location = value['result']
            for task, task_output in result_location.items():
                out = out_stub.copy()           
                out['task_type'] = task_type    
                out['task'] = task              
                out['data'] = task_output       
                outlist.append(out)
        for finding in app['assessment']['report']['findings']:
            out = out_stub.copy()
            out['task'] = 'FINDING'
            out['data'] = finding 
            outlist.append(out)
        out = out_stub.copy()
        out['task'] = 'NOWSECURE_SCORE'
        out['data'] = app['assessment']['report']['score']
        outlist.append(out)
    outfile_tokens = f.split('/')   
    outfilename = '/'.join(outfile_tokens[:-1]) + '/output/cleaned-' + outfile_tokens[-1] 
    print(outfilename)
    with open(outfilename,'w') as g:
        g.write(json.dumps(outlist)) 


/content/6f6f2eba-be28-4fb2-bf3c-1c80213dca01.json
/content/output/cleaned-6f6f2eba-be28-4fb2-bf3c-1c80213dca01.json
/content/6c2c8e54-1b9d-4715-b553-35314986eed1.json
/content/output/cleaned-6c2c8e54-1b9d-4715-b553-35314986eed1.json
/content/1ad17c50-f003-48eb-b4b9-7d9e859efbdb.json
/content/output/cleaned-1ad17c50-f003-48eb-b4b9-7d9e859efbdb.json
/content/7bb85a54-1b8a-43c6-82dd-b54b84b05fa9.json
/content/output/cleaned-7bb85a54-1b8a-43c6-82dd-b54b84b05fa9.json
/content/2f91f1e7-913d-4374-8d54-9b0e3b6bf917.json
/content/output/cleaned-2f91f1e7-913d-4374-8d54-9b0e3b6bf917.json
/content/1ac5fdbf-fd4e-4816-a0bc-795b3f0ed0cb.json
/content/output/cleaned-1ac5fdbf-fd4e-4816-a0bc-795b3f0ed0cb.json
/content/6b5c38fd-3799-4a19-8386-5352ef619122.json
/content/output/cleaned-6b5c38fd-3799-4a19-8386-5352ef619122.json
/content/3fe274e1-d178-426e-81fd-145e01f9ce1f.json
/content/output/cleaned-3fe274e1-d178-426e-81fd-145e01f9ce1f.json
/content/1c961800-6734-45f7-93ed-bb82296cf82c.json
/content/outp

In [ ]:
import pandas as pd

outlist = []
for filename in glob.glob('/content/output/cleaned-*.json'):
  outlist.extend(json.load(open(filename)))
df = pd.DataFrame(outlist)
print(df.shape)

(11672, 7)


In [ ]:
df.head()

,title,publisher,platformType,task_type,task,data,assessment
0,Homerun Battle 2,Com2uS,android,dynamic,fs_issues,"[{'fs_event': {'p': 4268, 'ts': 15676792897813...",NaN
1,Homerun Battle 2,Com2uS,android,dynamic,http_info,{'unique_user_agents': ['Dalvik/2.1.0 (Linux; ...,NaN
2,Homerun Battle 2,Com2uS,android,dynamic,ipc_issues,[],NaN
3,Homerun Battle 2,Com2uS,android,dynamic,file_access,"{'detailed_list': [{'p': 4268, 'ts': 156767928...",NaN
4,Homerun Battle 2,Com2uS,android,dynamic,run_summary,{'apk_info': {'md5': 'E66A455B8B6AEBF9C1C6BF71...,NaN


In [ ]:
# print first finding for the first app
doc[0]['assessment']['report']['findings'][0]

{'affected': True,
 'check': {'description': 'This test checks to see if the key used to sign the app is larger than 1024 bits. Anything less leaves your app vulnerable to forged digital signatures.'},
 'checkId': 'keysize_check',
 'cvss': 5.9,
 'impactType': 'medium'}

In [ ]:
findings = df[df['task'] == 'FINDING']
findings

,title,publisher,platformType,task_type,task,data,assessment
221,Homerun Battle 2,Com2uS,android,NaN,FINDING,"{'cvss': 7.5, 'impactType': 'high', 'affected'...",NaN
222,Homerun Battle 2,Com2uS,android,NaN,FINDING,"{'cvss': 7, 'impactType': 'high', 'affected': ...",NaN
223,Homerun Battle 2,Com2uS,android,NaN,FINDING,"{'cvss': 5.9, 'impactType': 'medium', 'affecte...",NaN
224,Homerun Battle 2,Com2uS,android,NaN,FINDING,"{'cvss': 4.6, 'impactType': 'medium', 'affecte...",NaN
225,Homerun Battle 2,Com2uS,android,NaN,FINDING,"{'cvss': 2.7, 'impactType': 'low', 'affected':...",NaN
...,...,...,...,...,...,...,...
11666,Interior Story: Design Home & Decorate Dream H...,DANKO GAMES EOOD,android,NaN,FINDING,"{'cvss': 0, 'impactType': 'pass', 'affected': ...",NaN
11667,Interior Story: Design Home & Decorate Dream H...,DANKO GAMES EOOD,android,NaN,FINDING,"{'cvss': 0, 'impactType': 'pass', 'affected': ...",NaN
11668,Interior Story: Design Home & Decorate Dream H...,DANKO GAMES EOOD,android,NaN,FINDING,"{'cvss': 0, 'impactType': 'pass', 'affected': ...",NaN
11669,Interior Story: Design Home & Decorate Dream H...,DANKO GAMES EOOD,android,NaN,FINDING,"{'cvss': 0, 'impactType': 'pass', 'affected': ...",NaN


In [ ]:
#findings['check_id'] = ''
#for index, row in findings.iterrows():
#  row['check_id'] = row['data']['checkId']

findings['cvss'] = findings['data'].apply(lambda x: x['cvss'])
findings['check_id'] = findings['data'].apply(lambda x: x['checkId'])

findings

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,title,publisher,platformType,task_type,task,data,assessment,cvss,check_id
221,Homerun Battle 2,Com2uS,android,NaN,FINDING,"{'cvss': 7.5, 'impactType': 'high', 'affected'...",NaN,7.5,android_target_sdk_min
222,Homerun Battle 2,Com2uS,android,NaN,FINDING,"{'cvss': 7, 'impactType': 'high', 'affected': ...",NaN,7.0,broken_ssl
223,Homerun Battle 2,Com2uS,android,NaN,FINDING,"{'cvss': 5.9, 'impactType': 'medium', 'affecte...",NaN,5.9,keysize_check
224,Homerun Battle 2,Com2uS,android,NaN,FINDING,"{'cvss': 4.6, 'impactType': 'medium', 'affecte...",NaN,4.6,allow_backup_check
225,Homerun Battle 2,Com2uS,android,NaN,FINDING,"{'cvss': 2.7, 'impactType': 'low', 'affected':...",NaN,2.7,secure_random_check
...,...,...,...,...,...,...,...,...,...
11666,Interior Story: Design Home & Decorate Dream H...,DANKO GAMES EOOD,android,NaN,FINDING,"{'cvss': 0, 'impactType': 'pass', 'affected': ...",NaN,0.0,okhttp
11667,Interior Story: Design Home & Decorate Dream H...,DANKO GAMES EOOD,android,NaN,FINDING,"{'cvss': 0, 'impactType': 'pass', 'affected': ...",NaN,0.0,potential_sqlcipher_key_leakage_check
11668,Interior Story: Design Home & Decorate Dream H...,DANKO GAMES EOOD,android,NaN,FINDING,"{'cvss': 0, 'impactType': 'pass', 'affected': ...",NaN,0.0,android_target_sdk_min
11669,Interior Story: Design Home & Decorate Dream H...,DANKO GAMES EOOD,android,NaN,FINDING,"{'cvss': 0, 'impactType': 'pass', 'affected': ...",NaN,0.0,certificate_validity_check


In [ ]:
app_df = findings[findings['check_id'] == 'apk_weak_crypto_hashing_algorithms']
app_df['weak_crypto'] = app_df['check_id'] == 'apk_weak_crypto_hashing_algorithms'
# This is OK, but would miss apps which didn't have a "apk_weak_crypto_hashing_algorithms" finding.

weak_crypto_apps = app_df['title'].tolist()
print(weak_crypto_apps)

['NJPW Collection', 'SkyAlert', 'PowerDirector - Video Editor, Video Maker', 'Kim Kardashian: Hollywood', 'BigOven: 1 Million+ Recipes and Meal Planner', 'Healthy Benefits Plus', 'Fair – The driver’s app', 'Dairyland®', 'RISK: Global Domination']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
outlist = []
for f in glob.glob('/content/*.json'):
  outlist.extend(json.load(open(f)))

app_df = pd.DataFrame(outlist)
print(app_df.shape)
app_df


(39, 3)


,title,latestAnalysisRecorded,assessment
0,Archery Go- Archery games & Archery,2020-05-25T04:11:05.688Z,{'analysis': {'task': {'dynamic': {'result': {...
1,Garfield Living Large!,2016-08-15T06:15:01.408Z,{'analysis': {'task': {'dynamic': {'result': {...
2,Web Browser & Explorer,2021-09-06T05:56:44.772Z,{'analysis': {'task': {'dynamic': {'result': {...
3,Birthday Card Maker,2020-06-05T14:58:22.466Z,{'analysis': {'task': {'dynamic': {'result': {...
4,Interior Story: Design Home & Decorate Dream H...,2021-05-23T02:29:11.937Z,{'analysis': {'task': {'dynamic': {'result': {...
5,Microsoft Excel,2021-09-03T08:46:41.547Z,{'analysis': {'task': {'dynamic': {'result': {...
6,Block craft sandbox: destruction simulator,2020-07-06T06:10:23.792Z,"{'analysis': {'task': {'dynamic': None, 'stati..."
7,Pro Piano Magic Tiles - Populer Songs,None,None
8,Saunders Comprehensive Review NCLEX-PN Examina...,2021-05-06T09:52:28.587Z,{'analysis': {'task': {'dynamic': {'result': {...
9,My Town: Play & Discover - City Builder Game,2020-12-24T08:23:50.965Z,"{'analysis': {'task': {'dynamic': None, 'stati..."


In [ ]:
app_df['weak_crypto'] = app_df['title'].isin(weak_crypto_apps)
app_df

#app_df['weak_crypto'] = False
#for index, row in app_df.iterrows():
#  prin()
#  row['weak_crypto'] = row['title'] in weak_crypto_apps

#app_df

,title,latestAnalysisRecorded,assessment,weak_crypto
0,Archery Go- Archery games & Archery,2020-05-25T04:11:05.688Z,{'analysis': {'task': {'dynamic': {'result': {...,False
1,Garfield Living Large!,2016-08-15T06:15:01.408Z,{'analysis': {'task': {'dynamic': {'result': {...,False
2,Web Browser & Explorer,2021-09-06T05:56:44.772Z,{'analysis': {'task': {'dynamic': {'result': {...,False
3,Birthday Card Maker,2020-06-05T14:58:22.466Z,{'analysis': {'task': {'dynamic': {'result': {...,False
4,Interior Story: Design Home & Decorate Dream H...,2021-05-23T02:29:11.937Z,{'analysis': {'task': {'dynamic': {'result': {...,False
5,Microsoft Excel,2021-09-03T08:46:41.547Z,{'analysis': {'task': {'dynamic': {'result': {...,False
6,Block craft sandbox: destruction simulator,2020-07-06T06:10:23.792Z,"{'analysis': {'task': {'dynamic': None, 'stati...",False
7,Pro Piano Magic Tiles - Populer Songs,None,None,False
8,Saunders Comprehensive Review NCLEX-PN Examina...,2021-05-06T09:52:28.587Z,{'analysis': {'task': {'dynamic': {'result': {...,False
9,My Town: Play & Discover - City Builder Game,2020-12-24T08:23:50.965Z,"{'analysis': {'task': {'dynamic': None, 'stati...",False


In [ ]:
app_df.iloc[4]

title                     Interior Story: Design Home & Decorate Dream H...
latestAnalysisRecorded                             2021-05-23T02:29:11.937Z
assessment                {'analysis': {'task': {'dynamic': {'result': {...
weak_crypto                                                           False
Name: 4, dtype: object

In [ ]:

# We're going to do the same pattern as Question 4 - initialize rows to something 
# empty, here -1.
app_df['cvss_min_score'] = -1
app_df['cvss_max_score'] = -1
app_df['cvss_avg_score'] = -1
app_df['cvss_count_score'] = -1

import numpy as np 
# We use Numpy to compute mean

app_df = app_df[app_df['title'] != 'Daily Quotes & Motivational']
# Iterate over the rows of the raw document
for index, row in app_df.iterrows():
  #app_info = row['assessment']
  #print(row.keys())
  if row['assessment'] is None:
    continue
  findings_list = row['assessment']['report']['findings']
  # Find the list of findings in the report.

  scores = [finding['cvss'] for finding in findings_list if finding['cvss'] > 0 ]
  print(len(findings_list))
  # Grab the scores from each of the findings. We're dropping the findings with 
  # cvss=3. It's not clear that's appropriate, but it's fine for illustrative 
  # purposes.

  name = row['title']
  # Grab the name to slice into the data frame

  print(scores)
  if len(scores) == 0:
    app_df.loc[app_df['title'] == name,'cvss_min_score'] = np.nan
    app_df.loc[app_df['title'] == name,'cvss_max_score'] = np.nan
    app_df.loc[app_df['title'] == name,'cvss_avg_score'] = np.nan
    app_df.loc[app_df['title'] == name,'cvss_count_score'] = 0
  else:
    app_df.loc[app_df['title'] == name,'cvss_min_score'] = min(scores)
    app_df.loc[app_df['title'] == name,'cvss_max_score'] = max(scores)
    app_df.loc[app_df['title'] == name,'cvss_avg_score'] = np.mean(scores)
    app_df.loc[app_df['title'] == name,'cvss_count_score'] = len(scores)
  # Put the appropriate functions of our scores list into their fields

app_df


122
[7.5, 7, 6.5, 4.7, 4.6, 2.3, 2.1, 2.1]
52
[7.5, 7.1, 6.5, 4.3]
124
[5.9, 4.6, 3.3, 2.7, 2.1]
122
[7.5, 6.5, 5, 4.7, 4.6, 2.3]
126
[5, 4.7, 4.6]
118
[5.3, 4, 3.7, 2.3]
16
[4.6]
128
[6.5, 5.7, 4.6]
18
[4.6]
125
[4.6, 2.7]
124
[]
126
[5.9, 4.6]
135
[5, 4.7, 4.6, 3.7, 3.7, 2.7]
18
[5.9]
122
[7.5, 7, 5.9, 4.6, 2.7, 2.3, 2.3, 1.8]
133
[5, 4.7, 3.7, 3.7]
133
[5.9, 5, 4.7, 3.7, 3.7, 3.3]
126
[4.6, 3.3]
121
[7.5, 4.7, 2.7, 2.1, 2.1]
135
[5.9, 4.6, 3.7, 3.7]
148
[7.5, 4.7, 2.7]
133
[3.7, 3.7, 2.5]
124
[2.7]
118
[5.3, 4.8, 4, 3.9, 3.7, 2.3, 2.3, 2.3]
135
[6.5, 5.9, 5, 4.7, 4.6, 3.7, 3.7, 2.7]
135
[5.9, 5, 4.7, 4.6, 3.7, 3.7]
123
[7.5]
125
[5.9, 4.6]
124
[3.3]
130
[3.3, 2.5]
124
[6.5, 6.1, 5.9, 5, 4.7, 4.6, 2.7, 2.3, 2.1]
131
[3.7, 3.7, 2.7]
122
[7.5, 7, 6.5, 2.7]
135
[5.9, 5, 4.7, 4.6, 3.7, 3.7]
124
[4.6]


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,title,latestAnalysisRecorded,assessment,weak_crypto,cvss_min_score,cvss_max_score,cvss_avg_score,cvss_count_score
0,Archery Go- Archery games & Archery,2020-05-25T04:11:05.688Z,{'analysis': {'task': {'dynamic': {'result': {...,False,2.1,7.5,4.600000,8
1,Garfield Living Large!,2016-08-15T06:15:01.408Z,{'analysis': {'task': {'dynamic': {'result': {...,False,4.3,7.5,6.350000,4
2,Web Browser & Explorer,2021-09-06T05:56:44.772Z,{'analysis': {'task': {'dynamic': {'result': {...,False,2.1,5.9,3.720000,5
3,Birthday Card Maker,2020-06-05T14:58:22.466Z,{'analysis': {'task': {'dynamic': {'result': {...,False,2.3,7.5,5.100000,6
4,Interior Story: Design Home & Decorate Dream H...,2021-05-23T02:29:11.937Z,{'analysis': {'task': {'dynamic': {'result': {...,False,4.6,5.0,4.766667,3
5,Microsoft Excel,2021-09-03T08:46:41.547Z,{'analysis': {'task': {'dynamic': {'result': {...,False,2.3,5.3,3.825000,4
6,Block craft sandbox: destruction simulator,2020-07-06T06:10:23.792Z,"{'analysis': {'task': {'dynamic': None, 'stati...",False,4.6,4.6,4.600000,1
7,Pro Piano Magic Tiles - Populer Songs,None,None,False,-1.0,-1.0,-1.000000,-1
8,Saunders Comprehensive Review NCLEX-PN Examina...,2021-05-06T09:52:28.587Z,{'analysis': {'task': {'dynamic': {'result': {...,False,4.6,6.5,5.600000,3
9,My Town: Play & Discover - City Builder Game,2020-12-24T08:23:50.965Z,"{'analysis': {'task': {'dynamic': None, 'stati...",False,4.6,4.6,4.600000,1


In [ ]:
app_df[['title','weak_crypto','cvss_min_score','cvss_max_score','cvss_avg_score','cvss_count_score']].to_csv('/content/analysis_df.csv')

In [ ]:
app_df.tail()

,title,latestAnalysisRecorded,assessment,weak_crypto,cvss_min_score,cvss_max_score,cvss_avg_score,cvss_count_score
34,SkyAlert,2021-05-17T17:23:03.876Z,{'analysis': {'task': {'dynamic': {'result': {...,True,2.7,3.7,3.366667,3
35,ParkDetroit,2020-06-08T10:48:38.073Z,{'analysis': {'task': {'dynamic': {'result': {...,False,2.7,7.5,5.925000,4
36,"PowerDirector - Video Editor, Video Maker",2021-09-06T09:52:00.640Z,{'analysis': {'task': {'dynamic': {'result': {...,True,3.7,5.9,4.600000,6
37,Currency Converter Plus Free with AccuRate™,2021-05-25T21:36:39.654Z,{'analysis': {'task': {'dynamic': {'result': {...,False,4.6,4.6,4.600000,1
38,Ramp Car Robot Transforming Game: Robot Car Games,None,None,False,-1.0,-1.0,-1.000000,-1


In [ ]:
app_df.columns

In [ ]:
app_df.to_csv('/content/app_df.csv', index=False)

In [ ]:
# Find only the findings where the app is affected.
true_findings = [i for i in doc[0]['assessment']['report']['findings'] if i['affected']]
print('%s total findings checked, with %s being true' % (len(doc[0]['assessment']['report']['findings']), len(true_findings)))

In [ ]:
#Question 1: 
print(len(doc))

In [ ]:
#Question 2:
#df[df['task'] == 'fs_issues'] #<- finding the fs_issues rows

# Iterate over the rows of the data frame where the task is "fs_issues". For each of these, 
# we're finding the list of fs_issues and checking if they have a 'wireable_executable' violation. 
# This code may have a bug - it seems right but found no matches in a trial data packet
for index, row in df[df['task'] == 'fs_issues'].iterrows():
  fs_issues_list = row['data']
  fs_issues_matching = len([i for i in fs_issues_list if i.get('violation','missing') == 'writeable_executable'])
  print(fs_issues_matching)

In [ ]:
# Question 3

# Define a function which looks through the snoop_network output and identifies 
# the ip address for each network connection, and returns them all as a list
def find_snoopnetwork_urls(objects):
  return [i['addr'] for i in objects['hosts'][0]['ip']]

# Below is a major pattern for iterating over a dataframe and wrangling the data.
# This pattern can be copy/pasted and then tweaked for a lot of purposes.

outlist = [] # generate a list of outputs. We'll put one dict per dataframe row into this.
for index, row in df[df['task'] == 'snoop_network'].iterrows():
  out = {} 
  # generate our output dict, which we'll fill up and append to outlist

  out['title'] = row['title'] 
  # save the app title - this is our key to match up the results to the app.

  out['urls'] = find_snoopnetwork_urls(row['data'])
  # Use the find_snoopnetwork_urls function above to grab the URLs

  out['num_of_urls'] = len(out['urls'])
  # Count the length of these URLs, and add that to the output as well

  outlist.append(out)
  # append the output dict for this row to the overall outlist

out_df = pd.DataFrame(outlist)
# Use pandas to turn the list-of-dicts into a data frame. Then print it.

out_df

In [ ]:
# Question 4

# We're going to start with the dataframe from Question 3, and then iterate over
# the main "doc" object, which contains not just the tasks but also the NowSecure
# "report" field

# doc[0]['assessment']['report']['score'] <- Score is here in the doc object
# doc[0]['title'] <- Title of app is here in the doc object

# This pattern will iterate over each app's info, and then slice the out_df 
# DataFrame from Question 3 to update the score with the data from the report
out_df['score'] = -1
for app_info in doc:
  score = app_info['assessment']['report']['score']
  name = app_info['title']
  out_df.loc[out_df['title'] == name,'score'] = score
out_df


In [ ]:
# Question 5
#doc[0]['assessment']['report']['findings'] <- This is the location of findings

# We're going to do the same pattern as Question 4 - initialize rows to something 
# empty, here -1.
out_df['cvss_min_score'] = -1
out_df['cvss_max_score'] = -1
out_df['cvss_avg_score'] = -1
out_df['cvss_count_score'] = -1

import numpy as np 
# We use Numpy to compute mean

# Iterate over the rows of the raw document
for app_info in doc:
  findings_list = app_info['assessment']['report']['findings']
  # Find the list of findings in the report.

  scores = [finding['cvss'] for finding in findings_list if finding['cvss'] > 0 ]
  # Grab the scores from each of the findings. We're dropping the findings with 
  # cvss=3. It's not clear that's appropriate, but it's fine for illustrative 
  # purposes.

  name = app_info['title']
  # Grab the name to slice into the data frame

  out_df.loc[out_df['title'] == name,'cvss_min_score'] = min(scores)
  out_df.loc[out_df['title'] == name,'cvss_max_score'] = max(scores)
  out_df.loc[out_df['title'] == name,'cvss_avg_score'] = np.mean(scores)
  out_df.loc[out_df['title'] == name,'cvss_count_score'] = len(scores)
  # Put the appropriate functions of our scores list into their fields

out_df

  
